In [1]:
import findspark 
findspark.init() 

import pyspark
from delta import *

In [2]:
spark = pyspark.sql.SparkSession.builder.master("spark://spark-master:7077").appName("spark_kafka").getOrCreate()

sc = spark.sparkContext
sc.setLogLevel('ERROR')

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /opt/spark/cache
The jars for the packages stored in: /opt/spark/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
com.mysql#mysql-connector-j added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-20c5a146-6320-4bea-905e-75cb268ec6e5;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.2.0 in central
	found io.delta#delta-storage;2.2.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.ap

23/02/21 23:41:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
df = spark\
    .readStream\
    .format("kafka")\
    .option("kafka.bootstrap.servers", "kafka-broker:9092")\
    .option("subscribe", "test-topic-1")\
    .option("startingOffsets", "earliest") \
    .load()

In [6]:
# df.selectExpr("cast(value as string) as payload", "*").limit(2).toPandas()

In [7]:
df\
.selectExpr("key", "topic", "partition", "offset", "timestamp", "timestampType", "cast(value as string) as payload") \
.writeStream \
.format("delta") \
.outputMode("append") \
.option("checkpointLocation", "/opt/spark/work-dir/odp_intra_storage/spark/datalake/test_db.db/spark_kafka").toTable("test_db.spark_kafka")

In [14]:
spark.sql("select current_timestamp - INTERVAL 10 minutes").toPandas()

/opt/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,current_timestamp() - INTERVAL '10' MINUTE
0,2023-02-22 00:03:04.169572


In [15]:
spark.sql("OPTIMIZE test_db.spark_kafka").toPandas()

,path,metrics
0,file:/opt/spark/work-dir/odp_intra_storage/spa...,"(1, 1201, (43947, 43947, 43947.0, 1, 43947), (..."


In [17]:
spark.sql("set spark.databricks.delta.retentionDurationCheck.enabled = false")
spark.sql("VACUUM test_db.spark_kafka RETAIN 0 HOURS").toPandas()

Deleted 1405 files and directories in a total of 5 directories.


,path
0,file:/opt/spark/work-dir/odp_intra_storage/spa...
